# Candlesticks patterns and statistical analysis
# Loading and preparing data

In [1]:
import pandas as pd
df = pd.read_csv("USDJPY_Candlestick_1_D_ASK_05.05.2003-19.10.2019.csv")
df.tail()

,Local time,open,high,low,close,volume
4323,14.10.2019 00:00:00.000 GMT+0300,108.372,108.525,108.033,108.404,122267.0400
4324,15.10.2019 00:00:00.000 GMT+0300,108.408,108.901,108.157,108.868,161800.8200
4325,16.10.2019 00:00:00.000 GMT+0300,108.878,108.878,108.565,108.792,178244.1800
4326,17.10.2019 00:00:00.000 GMT+0300,108.793,108.941,108.454,108.692,234278.7681
4327,18.10.2019 00:00:00.000 GMT+0300,108.706,108.724,108.388,108.433,148743.3600


In [2]:
#Check if any zero volumes are available
indexZeros = df[ df['volume'] == 0 ].index

df.drop(indexZeros , inplace=True)
df.loc[(df["volume"] == 0 )]
df.isna().sum()

Local time    0
open          0
high          0
low           0
close         0
volume        0
dtype: int64

In [3]:
import numpy as np
import pandas_ta as ta
#df.ta.indicators()
#help(ta.atr)
df['ATR'] = df.ta.atr(length=10)
df['RSI'] = df.ta.rsi()
df.dropna()

,Local time,open,high,low,close,volume,ATR,RSI
14,23.05.2003 00:00:00.000 GMT+0300,117.223,117.433,116.740,116.871,564114.1005,1.147194,38.950893
15,26.05.2003 00:00:00.000 GMT+0300,116.813,117.387,116.669,116.841,565039.0928,1.104275,38.795462
16,27.05.2003 00:00:00.000 GMT+0300,116.836,117.317,116.180,117.279,574499.0030,1.107547,42.408832
17,28.05.2003 00:00:00.000 GMT+0300,117.283,118.900,116.933,118.689,566670.9004,1.193493,52.193486
18,29.05.2003 00:00:00.000 GMT+0300,118.680,119.144,117.903,118.272,564631.6944,1.198243,49.514189
...,...,...,...,...,...,...,...,...
4323,14.10.2019 00:00:00.000 GMT+0300,108.372,108.525,108.033,108.404,122267.0400,0.683948,61.010164
4324,15.10.2019 00:00:00.000 GMT+0300,108.408,108.901,108.157,108.868,161800.8200,0.689953,65.223022
4325,16.10.2019 00:00:00.000 GMT+0300,108.878,108.878,108.565,108.792,178244.1800,0.652258,64.003170
4326,17.10.2019 00:00:00.000 GMT+0300,108.793,108.941,108.454,108.692,234278.7681,0.635732,62.350755


In [4]:
# for down trends
# Shooting star

def Revsignal1(df1):
    df.dropna()
    df.reset_index(drop=True, inplace=True)
    
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    signal = [0] * length
    highdiff = [0] * length
    lowdiff = [0] * length
    bodydiff = [0] * length
    ratio1 = [0] * length
    ratio2 = [0] * length
        
    for row in range(0, length):
        
        highdiff[row] = high[row]-max(open[row],close[row])
        bodydiff[row] = abs(open[row]-close[row])
        if bodydiff[row]<0.002:
            bodydiff[row]=0.002
        lowdiff[row] = min(open[row],close[row])-low[row]
        ratio1[row] = highdiff[row]/bodydiff[row]
        ratio2[row] = lowdiff[row]/bodydiff[row]
        #print(df.RSI[row])
        if (ratio1[row]>2.5 and lowdiff[row]<0.3*highdiff[row] and bodydiff[row]>0.03 and df.RSI[row]>50 and df.RSI[row]<70 ):
            signal[row] = 1
        elif (ratio2[row-1]>2.5 and highdiff[row-1]<0.23*lowdiff[row-1] and bodydiff[row-1]>0.03 and bodydiff[row]>0.04 and close[row]>open[row] and close[row]>high[row-1] and df.RSI[row]<55 and df.RSI[row]>30):
            signal[row] = 2
        #elif (ratio2[row]>2.5 and highdiff[row]<0.23*lowdiff[row] and bodydiff[row]>0.03 and df.RSI[row]<70):
        #    signal[row]=2
    return signal
df['signal1'] = Revsignal1(df)
df[df['signal1']==1].count()

Local time    36
open          36
high          36
low           36
close         36
volume        36
ATR           36
RSI           36
signal1       36
dtype: int64

In [5]:
#Target Shooting Star

def mytarget(barsupfront, df1):
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    datr = list(df1['ATR'])
    trendcat = [None] * length
    

    for line in range (0,length-barsupfront-1):
        valueOpenLow = 0
        valueOpenHigh = 0
        
        highdiff = high[line]-max(open[line],close[line])
        bodydiff = abs(open[line]-close[line])
        
        pipdiff = datr[line]*1. #highdiff*1.3 #for SL 400*1e-3
        if pipdiff<1.1:
            pipdiff=1.1
            
        SLTPRatio = 2. #pipdiff*Ratio gives TP
        
        for i in range(1,barsupfront+1):
            value1 = close[line]-low[line+i]
            value2 = close[line]-high[line+i]
            valueOpenLow = max(value1, valueOpenLow)
            valueOpenHigh = min(value2, valueOpenHigh)

            if ( (valueOpenLow >= (SLTPRatio*pipdiff) ) and (-valueOpenHigh < pipdiff) ):
                trendcat[line] = 1 #-1 downtrend
                break
            elif ((valueOpenLow < pipdiff) ) and (-valueOpenHigh >= (SLTPRatio*pipdiff)):
                trendcat[line] = 2 # uptrend
                break 
            else:
                trendcat[line] = 0 # no clear trend
            
    return trendcat

# mytarget(barsfront to take into account, dataframe)
df['Trend'] = mytarget(100, df)
#df.head(10)

In [6]:
conditions = [(df['Trend'] == 1) & (df['signal1'] == 1),
              (df['Trend'] == 2) & (df['signal1'] == 2)]
values = [1, 2]
df['result'] = np.select(conditions, values)

trendId=1
print(df[df['result']==trendId].result.count()/df[df['signal1']==trendId].signal1.count())
df[ (df['result']!=trendId) & (df['signal1']==trendId) ] # false positives

#histrs = df[ (df['result']==2) & (df['signal1']==2) ].RSI # false positives
#import matplotlib.pyplot as plt
#plt.hist(histrs,bins=2)  # density=False would make counts
#plt.ylabel('Probability')
#plt.xlabel('RSI');

0.5833333333333334


,Local time,open,high,low,close,volume,ATR,RSI,signal1,Trend,result
186,20.01.2004 00:00:00.000 GMT+0200,107.229,107.900,106.952,107.072,6.082301e+05,0.790313,50.361965,1,0.0,0
580,25.07.2005 00:00:00.000 GMT+0300,111.347,111.998,111.210,111.508,6.041630e+05,1.126688,53.553068,1,0.0,0
643,20.10.2005 00:00:00.000 GMT+0300,115.440,115.767,115.263,115.325,5.879348e+05,0.846232,66.012856,1,2.0,0
760,03.04.2006 00:00:00.000 GMT+0300,117.805,118.790,117.591,117.695,5.831051e+05,1.069075,52.196782,1,0.0,0
1048,10.05.2007 00:00:00.000 GMT+0300,120.100,120.550,119.850,119.920,3.273105e+06,0.704032,57.089153,1,2.0,0
1302,30.04.2008 00:00:00.000 GMT+0300,104.040,104.895,103.700,103.905,2.509382e+05,1.230512,57.856887,1,0.0,0
1307,07.05.2008 00:00:00.000 GMT+0300,104.770,105.600,104.615,104.735,2.909724e+05,1.164444,59.973593,1,0.0,0
1824,30.04.2010 00:00:00.000 GMT+0300,94.040,94.590,93.850,93.885,2.948239e+05,0.841398,59.232221,1,0.0,0
2282,25.01.2012 00:00:00.000 GMT+0200,77.673,78.287,77.567,77.785,1.092068e+05,0.404107,64.030737,1,0.0,0
3086,24.02.2015 00:00:00.000 GMT+0200,118.818,119.842,118.767,118.986,1.119961e+05,1.019934,52.287097,1,2.0,0


In [7]:
dfpl = df[550:650]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.show()

## Live Automated Orders with email notification

In [8]:
from apscheduler.schedulers.blocking import BlockingScheduler
import json
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleCollector, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

token = "XXXXXXXX"#you need token here generated from OANDA account

import smtplib
gmail_user = 'XXXXXXXX'
gmail_password = 'XXXXXX'
sent_from = gmail_user
to = ['XXXXXEMAILXXXX']
subject = 'info JPY'

In [9]:
def some_job():
    access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    #collector = CandleCollector(access_token, Pair.USD_JPY, Gran.D)
    collector = client.get_collector(Pair.USD_JPY, Gran.D)
    candles = collector.grab(2*161)

    dfstream = pd.DataFrame(columns=['open','close','high','low'])
    i=0
    for candle in candles:
        dfstream.loc[i, ['open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['high']] = float(str(candle.bid.h))
        dfstream.loc[i, ['low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['open'] = dfstream['open'].astype(float)
    dfstream['close'] = dfstream['close'].astype(float)
    dfstream['high'] = dfstream['high'].astype(float)
    dfstream['low'] = dfstream['low'].astype(float)

   
    import numpy as np
    import pandas_ta as ta
    dfstream['ATR'] = dfstream.ta.atr(length=20)
    dfstream['RSI'] = dfstream.ta.rsi()

    #________________________________________________________________________________________________
    X_stream = dfstream.iloc[[319,320]].copy()# !!! Index takes last CLOSED candle and the one before
    X_stream.reset_index(drop=True, inplace=True)
    signal = 0
    length = len(X_stream)

    highdiff = [0] * length
    lowdiff = [0] * length
    bodydiff = [0] * length
    ratio1 = [0] * length
    ratio2 = [0] * length
   
    #print(X_stream.high[0])

    for row in range(0,length):
        highdiff[row] = X_stream.high[row]-max(X_stream.open[row],X_stream.close[row])
        bodydiff[row] = abs(X_stream.open[row]-X_stream.close[row])
        if bodydiff[row]<0.002:
            bodydiff[row]=0.002
        lowdiff[row] = min(X_stream.open[row],X_stream.close[row])-X_stream.low[row]
        ratio1[row] = highdiff[row]/bodydiff[row]
        ratio2[row] = lowdiff[row]/bodydiff[row]

    row=1
    if (ratio1[row]>2.5 and lowdiff[row]<0.3*highdiff[row] and bodydiff[row]>0.03 and X_stream.RSI[row]>50 and X_stream.RSI[row]<70 ):
        signal = 1
    elif (ratio2[row-1]>2.5 and highdiff[row-1]<0.23*lowdiff[row-1] and bodydiff[row-1]>0.03 and bodydiff[row]>0.04 and X_stream.Close[row]>X_stream.open[row] and X_stream.close[row]>X_stream.high[row-1] and X_stream.RSI[row]<55 and X_stream.RSI[row]>30):
        signal = 2
    else:
        signal = 0
    
    
    #------------------------------------
    # send email with 
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    msg = str(signal)+" USDJPY  "
    server.sendmail(sent_from, to, msg)
    #________________________________________________________________________________________________
    
    # EXECUTING ORDERS
    accountID = "XXXXXXX" #your account ID here
    access_token='XXXXX'#access token here
    client = API(access_token)
    
    candles = collector.grab(1)
    for candle in candles:
        print(candle.bid.o)
        print(candle.bid.c)
        print(candle.bid.h)
        print(candle.bid.l)
    
    pipdiff = X_stream.ATR[row]*1. #highdiff*1.3 #for SL 400*1e-3
    if pipdiff<1.1:
        pipdiff=1.1
            
    SLTPRatio = 2. #pipdiff*Ratio gives TP
    
    TPBuy = float(str(candle.bid.o))+pipdiff*SLTPRatio
    SLBuy = float(str(candle.bid.o))-pipdiff
    TPSell = float(str(candle.bid.o))-pipdiff*SLTPRatio
    SLSell = float(str(candle.bid.o))+pipdiff
    
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="USD_JPY", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="USD_JPY", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

## Executing orders automatically every Day

In [10]:
#some_job()

###################################################################
## Interval time job ##############################################
scheduler = BlockingScheduler(job_defaults={'misfire_grace_time': 15*60})
#scheduler = BlockingScheduler()
scheduler.add_job(some_job, 'cron', day_of_week='mon-fri', hour='*/23', minute=59, jitter=120, timezone='America/New_York')
#scheduler.add_job(some_job, 'cron', day_of_week='mon-fri', hour='*/4', minute=5, jitter=120, timezone='America/New_York')
#scheduler.add_job(some_job, 'interval', seconds=5)
scheduler.start()